In [1]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [41]:
data = pd.read_csv('./data/sample_music')
data.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop,song_popularity
0,0,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,...,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4,Like a Rolling Stone,Bob Dylan,"['classic rock', 'country rock', 'folk', 'folk...",72,68
1,1,0.485,0.863,1,-9.027,1,0.0495,0.000012,0.016200,0.1380,...,spotify:track:3oTlkzk1OtrhH8wBAduVEi,https://api.spotify.com/v1/tracks/3oTlkzk1Otrh...,https://api.spotify.com/v1/audio-analysis/3oTl...,300977,4,Smells Like Teen Spirit,Nirvana,"['grunge', 'permanent wave', 'rock']",82,0
2,2,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,...,spotify:track:3ZFBeIyP41HhnALjxWy1pR,https://api.spotify.com/v1/tracks/3ZFBeIyP41Hh...,https://api.spotify.com/v1/audio-analysis/3ZFB...,337413,4,A Day In The Life - Remastered,The Beatles,"['beatlesque', 'british invasion', 'classic ro...",84,0
3,3,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,...,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...,219147,4,Good Vibrations (Mono),The Beach Boys,"['baroque pop', 'classic rock', 'psychedelic r...",78,20
4,4,0.518,0.756,10,-10.851,1,0.0915,0.735000,0.000062,0.3170,...,spotify:track:7MH2ZclofPlTrZOkPzZKhK,https://api.spotify.com/v1/tracks/7MH2ZclofPlT...,https://api.spotify.com/v1/audio-analysis/7MH2...,160893,4,Johnny B Goode,Chuck Berry,"['classic rock', 'rock', 'rock-and-roll', 'roc...",75,0


In [42]:
data.drop(columns='Unnamed: 0',inplace=True)

In [43]:
#function to turn artist genres string to list
def genre_list(self):
    split = self.strip().split(',')
    clean = [x.strip(" [']") for x in split]
    return clean

In [44]:
# convert the df to list
data['artist_genres'] = data['artist_genres'].apply(lambda x: genre_list(x))

In [60]:
#finding all the genres in the data set
all_genres_counts = {}
for row in data.artist_genres:
    for x in row:
        if x not in all_genres_counts:
            all_genres_counts[x] = 1
        else:
            all_genres_counts[x] += 1
        
print(all_genres_counts)
    

{'classic rock': 301, 'country rock': 65, 'folk': 63, 'folk rock': 121, 'rock': 455, 'roots rock': 58, 'singer-songwriter': 71, 'grunge': 12, 'permanent wave': 185, 'beatlesque': 54, 'british invasion': 74, 'merseybeat': 47, 'psychedelic rock': 129, 'baroque pop': 28, 'sunshine pop': 17, 'rock-and-roll': 94, 'rockabilly': 65, 'classic soul': 66, 'jazz blues': 18, 'memphis soul': 22, 'soul': 127, 'southern soul': 30, 'adult standards': 83, 'brill building pop': 9, 'classic girl group': 14, 'funk': 72, 'indie r&b': 7, 'motown': 37, 'northern soul': 10, 'quiet storm': 43, 'pop': 55, 'r&b': 24, 'heartland rock': 17, 'mellow gold': 123, 'album rock': 102, 'art rock': 145, 'blues rock': 71, 'hard rock': 62, 'alternative rock': 122, 'madchester': 26, 'new wave': 79, 'post-punk': 40, 'uk post-punk': 21, 'soul blues': 14, 'atl hip hop': 14, 'dirty south rap': 6, 'hip hop': 68, 'old school atlanta hip hop': 3, 'pop rap': 16, 'rap': 70, 'southern hip hop': 10, 'punk': 47, 'dance pop': 65, 'indiet

In [46]:
data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop,song_popularity
0,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,0.557,...,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4,Like a Rolling Stone,Bob Dylan,"[classic rock, country rock, folk, folk rock, ...",72,68
1,0.485,0.863,1,-9.027,1,0.0495,0.000012,0.016200,0.1380,0.767,...,spotify:track:3oTlkzk1OtrhH8wBAduVEi,https://api.spotify.com/v1/tracks/3oTlkzk1Otrh...,https://api.spotify.com/v1/audio-analysis/3oTl...,300977,4,Smells Like Teen Spirit,Nirvana,"[grunge, permanent wave, rock]",82,0
2,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,0.175,...,spotify:track:3ZFBeIyP41HhnALjxWy1pR,https://api.spotify.com/v1/tracks/3ZFBeIyP41Hh...,https://api.spotify.com/v1/audio-analysis/3ZFB...,337413,4,A Day In The Life - Remastered,The Beatles,"[beatlesque, british invasion, classic rock, m...",84,0
3,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,0.331,...,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...,219147,4,Good Vibrations (Mono),The Beach Boys,"[baroque pop, classic rock, psychedelic rock, ...",78,20
4,0.518,0.756,10,-10.851,1,0.0915,0.735000,0.000062,0.3170,0.968,...,spotify:track:7MH2ZclofPlTrZOkPzZKhK,https://api.spotify.com/v1/tracks/7MH2ZclofPlT...,https://api.spotify.com/v1/audio-analysis/7MH2...,160893,4,Johnny B Goode,Chuck Berry,"[classic rock, rock, rock-and-roll, rockabilly]",75,0


In [49]:
#combining genres with music feature data average into a new df
genre_features_input = {}
genre_features = {}
#list of features were looking for
features_input = ['danceability','energy','loudness','speechiness','acousticness','liveness','valence']


#loop through df
for i, r in data.iterrows():

    #loop through each genre
    for genre in r['artist_genres']:

        #create a dict for the genre if it isnt made
        if genre not in genre_features_input.keys():
            genre_features_input[genre] = []

            #start adding the features
            for feature in features_input:

                #create dict for features
                if feature not in genre_features.keys():
                    genre_features[feature] = []
                    genre_features[feature].append(r[feature])
                
            genre_features_input[genre].append(genre_features)
        #else add onto the generated genre key
        else:
            for genre_info in genre_features_input.values():
                for dicts in genre_info:
                    for key,v in dicts.items():
                        for feature in features_input:
                            if key == feature:
                                dicts[key].append(r[feature])
                    
                    
                
        genre_features = {}
        


In [50]:
genre_test = pd.DataFrame(genre_features_input)
genre_test.head()

,classic rock,country rock,folk,folk rock,rock,roots rock,singer-songwriter,grunge,permanent wave,beatlesque,...,makossa,vocal harmony group,ambient house,melodic rap,mississippi hip hop,indie punk,riot grrrl,bakersfield sound,bubblegrunge,country boogie
0,"{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.482, 0.485, 0.364, 0.364, ...","{'danceability': [0.485, 0.485, 0.364, 0.364, ...","{'danceability': [0.485, 0.485, 0.364, 0.364, ...","{'danceability': [0.364, 0.364, 0.364, 0.398, ...",...,"{'danceability': [0.831, 0.831, 0.504, 0.504, ...","{'danceability': [0.557, 0.682, 0.682, 0.682, ...","{'danceability': [0.682, 0.682, 0.682, 0.682, ...","{'danceability': [0.794, 0.794, 0.794, 0.794, ...","{'danceability': [0.794, 0.794, 0.794, 0.794, ...","{'danceability': [0.435, 0.435, 0.725, 0.483, ...","{'danceability': [0.435, 0.725, 0.483, 0.483, ...","{'danceability': [0.725, 0.725, 0.483, 0.483, ...","{'danceability': [0.483, 0.483, 0.483, 0.763, ...","{'danceability': [0.699, 0.699, 0.699, 0.747, ..."


In [51]:
#get the average of all the variables that was gathered from each artists songs associated with the genre
for genre_key in genre_features_input.values():
    for item in genre_key:
        for k,v in item.items():
            # print(v[:10])
            new = np.mean(v)
            item[k] = new
            


In [52]:
genre_features_input['acid rock']

[{'danceability': 0.5574073906758222,
  'energy': 0.6312868305023491,
  'loudness': -9.093789663895915,
  'speechiness': 0.06562278641127575,
  'acousticness': 0.27453364559269966,
  'liveness': 0.19407294904228403,
  'valence': 0.5901399710878208}]

In [64]:
genre_name = []
danceability = []
energy = []
loudness = []
speechiness = []
acousticness = []
liveness = []
valence = []
genres_counts = []

for key, value in genre_features_input.items():
    genre_name.append(key)
    for k, v in value[0].items():
        if k == 'danceability':
            danceability.append(v)
        elif k == 'energy':
            energy.append(v)
        elif k == 'loudness':
            loudness.append(v)
        elif k == 'speechiness':
            speechiness.append(v)
        elif k == 'acousticness':
            acousticness.append(v)
        elif k == 'liveness':
            liveness.append(v)
        else:
            valence.append(v)
for k, v in all_genres_counts.items():
        genres_counts.append(v)

In [65]:
test_dict = {'genre_name': genre_name,
'danceability': danceability,
'energy': energy,
'loudness': loudness,
'speechiness': speechiness,
'acousticness': acousticness,
'liveness': liveness,
'valence': valence,
'genre_counts': genres_counts}
genre_test = pd.DataFrame(test_dict)

In [66]:
genre_test

,genre_name,danceability,energy,loudness,speechiness,acousticness,liveness,valence,genre_counts
0,classic rock,0.556515,0.631028,-9.100323,0.065706,0.275440,0.193187,0.590737,301
1,country rock,0.556515,0.631028,-9.100323,0.065706,0.275440,0.193187,0.590737,65
2,folk,0.556515,0.631028,-9.100323,0.065706,0.275440,0.193187,0.590737,63
3,folk rock,0.556515,0.631028,-9.100323,0.065706,0.275440,0.193187,0.590737,121
4,rock,0.556515,0.631028,-9.100323,0.065706,0.275440,0.193187,0.590737,455
...,...,...,...,...,...,...,...,...,...
471,indie punk,0.610125,0.647937,-7.843422,0.046425,0.271573,0.158836,0.642484,1
472,riot grrrl,0.612905,0.643317,-7.906889,0.045400,0.275690,0.160383,0.643810,1
473,bakersfield sound,0.617508,0.638794,-7.945587,0.044605,0.287845,0.160629,0.645397,1
474,bubblegrunge,0.616167,0.637350,-8.041883,0.044283,0.276227,0.164397,0.650800,1


In [68]:
top10_genres = genre_test.nlargest(10, 'genre_counts')

fig = px.bar(top10_genres, x='genre_name', y=['valence', 'energy', 'danceability', 'acousticness'], barmode='group')


fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.2
    )
)
fig.show()